In [1]:
%matplotlib inline 
from IPython.display import display, HTML
import pandas as pd 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
init_notebook_mode(connected=True)
import os 
import numpy as np 
import pylab as pl
import scipy.stats as st
import glob 
from ifis_tools import database_tools as db
from ifis_tools import series_tools as ser
import hydroeval as heval

In [12]:
def Read_Qobs(linkID):
    for k in USGS.keys():
        if USGS[k] == linkID:
            USGS_id = k
    Qobs = db.SQL_read_USGS_Streamflow(USGS_id, '2008-1-1','2018-12-31')
    return Qobs['val']
def MedianByInterval(RCobs, RCsim, bins = np.linspace(0,1,6)):
    Ps = [0]
    X = [0]
    for b1,b2 in zip(bins[:-1],bins[1:]):
        pos = np.where((RCobs>b1)&(RCobs<=b2))[0]
        if pos.size>0:
            Ps.append(np.percentile(RCsim[pos], 50))
        else:
            Ps.append(np.nan)
        X.append((b1+b2)/2.)
    return np.array(Ps), np.array(X)

# Tl Analysis.

Here we perform an analysis of $T_l$ vrying its value on the model and evaluating the performance on the model.  Steps follow:

1. Run HLM model with different values of $T_l$:
    - 0.1, 0.09, 0.07, 0.05, 0.03
2. For all links with USGS info, read the data and compare results in terms of:
    - RCobs Vs RCsim dispersion.
    - Nash eff of events.
    - KG eff of events.

In [14]:
path = '/media/nicolas/Data/RunoffExp/Results/'

In [10]:
USGS = db.SQL_USGS_at_IFIS()
Links = [USGS[k] for k in USGS.keys()]

In [18]:
Qobs = {}
Qsim = {}
Events = {}

In [ ]:
link = Links[25]
#Read the data
qobs = Read_Qobs(link)
Qhlm = pd.read_msgpack(path + 'HLM190/'+str(link)+'_190.msg')
Rain = pd.read_msgpack(path + 'Rainfall/Rain_'+str(link)+'.msg')
#Read data and separate flux
qobs, Qrun, QsepO = ser.Runoff_SeparateBaseflow(qobs, Qhlm)
#Mean anual maximum 
Qmax = qobs.resample('A').max().mean()
Area = db.SQL_Get_linkArea(link)
#Find events and estimates observed RC
pos1, pos2 = ser.Runoff_FindEvents(qobs, Qrun, minPeak=Qmax*0.2)
RCobs = ser.Runoff_CompleteAnalysis(Area, qobs, Rain, QsepO, pos1, pos2, N = pd.Timedelta('2d'))
#Updates observed dictionary
Qobs.update({str(link): {'Q':qobs, 'BR': QsepO, 'RC': RCobs['RC'], 'Rain': RCobs['RainEvent']}})
#Updates dictionary with events 
Events.update({str(link): {'p1': pos1, 'p2':pos2}})
#Read the model 
QsimT = {}
for config in ['01','003','005','007','012']:
    #read data and separate baseflow
    qs = pd.read_msgpack(path + 'HLM254/'+str(link)+'_'+config+'.msg')
    qs, Qrun, qsepS = ser.Runoff_SeparateBaseflow(qs, Qhlm)
    #Estimates RunofLf 
    RC = ser.Runoff_CompleteAnalysis(Area, qs, Rain, qsepS, pos1, pos2, N = pd.Timedelta('2d'))
    #Update dictionary with simulations and separated baseflow
    QsimT.update({config: {'Q':qs, 'BR':qsepS, 'RC': RC['RC']}})
Qsim.update({str(link): QsimT})

## Voy en que tengo que copiar lo que no sirve a Old (o que bobada?)

In [28]:
Good = []
for l in Links:
    qobs = Read_Qobs(l)
    if qobs.size > 0:
        Good.append(l)
        qobs.to_msgpack('/media/nicolas/Data/RunoffExp/Results/USGS/'+str(link)+'.msg')
    else:
        

In [36]:
path = '/media/nicolas/Data/RunoffExp/Results/'

In [68]:
for k in glob.iglob(path +'**/' +str(link)+'*', recursive=True):
    comando = 'mv '+k+' '+os.path.dirname(k)+'/noObs/'

In [69]:
comando

'mv /media/nicolas/Data/RunoffExp/Results/HLM254/216095_009.msg /media/nicolas/Data/RunoffExp/Results/HLM254/noObs/'